# Columnas inútiles y valores sin sentido

Cuando realizamos el **AED** en el tema anterior, vimos cómo podíamos eliminar columnas que no íbamos a utilizar de nuestro conjunto de datos.

En particular, vimos lo sencillo que era hacerlo desde ```pandas```. Durante este cuadernillo vamos a ir un poco más allá y vamos a eliminar, además de las columnas, los datos sin sentido.

**MUY IMPORTANTE**: un dato sin sentido es un fallo de métrica a la hora de recoger el dato. Por ejemplo:
- Tener 5 millones de grados centígrados en La Tierra.
- Tener -6 años.
- Llamarse 76126.
- Ser el día Farola de la semana.

No tiene nada que ver con los valores atípicos, que son valores que tienen sentido, pero son muy poco comunes. Por ejemplo:
- Poner el horno a 400ºC.
- Tener 120 años.
- Que alguien diga que su día favorito es el lunes (sí, hay gente muy rara por el mundo).

## Las columnas inútiles para el ```dataset``` Titanic

Durante este cuadernillo, trabajaremos sobre el conjunto de datos Titanic, que nos muestra la información de las personas que viajaban en este barco cuando se hundió.

In [1]:
import seaborn as sns

df = sns.load_dataset("titanic")
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


Determinar si una columna es útil o no es una tarea relativamente compleja: no suele depender del ```dataset``` en sí mismo, sino de la tarea que queramos realizar con él.

Por ejemplo, para un **AED**, eliminaremos la mínima cantidad de información posible, dado que queremos _explorar_ nuestro conjunto de datos.

Sin embargo, si queremos clasificar la clase en la que viajaba la persona, podremos eliminar columnas inútiles, como si murió o no.

Además, tendremos que tener especial cuidado a la hora de borrar estas columnas, dado que a veces los datos pueden ser antiintuitivos y depender de una columna que no aparenta tener nada en relación con el resultado.

En general, podemos hablar de **tres** formas de borrar la información:
- **Información irrelevante o indeseada**: el más fácil de entender, simplemente no nos interesa esa información o nos dicen expresamente que no podemos utilizarla para el análisis.
- **Información recalculada**: información que de alguna forma utiliza otras columnas (estos valores pueden ser detectados con un análisis de correlación de variables). A simple vista, en nuestro ejemplo, podríamos entender que las variables ```age```, ```sex``` y ```adult_male``` están correladas.
- **Información que introduce sesgos**: ante sucesos pasados, puede ser muy relevante saber cierta información de las personas. Sin embargo, si quisiésemos predecir lo que pudiera pasar en un futuro, deberíamos eliminar información de sesgo que puede introducir un resultado parcial. Por ejemplo, quizá la ciudad de embarque o el sexo de la persona puede introducir sesgos a la hora de hablar de accidentes de barco. _¿Te imaginas que solo por embarcar en España o ser mujer tuvieses un 20% más de probabilidad de sufrir un naufragio?_

Cuando tengamos cualquiera de estos tres casos, simplemente borraremos esa información (la clasificación anterior, en general, nos servirá para argumentar ese borrado). Por ejemplo: _elimino la variable ```sexo``` para este análisis porque considero que introduce un sesgo sobre la información._

En particular, veamos cuáles son las columnas de nuestro ```dataset``` Titanic.

In [2]:
df.columns

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town',
       'alive', 'alone'],
      dtype='object')

Supongamos que nuestro objetivo es determinar si una persona cualquiera sobrevivió al incidente. Obviamente:
- ```survived``` determina si sobrevivió. La necesitamos (¡es la etiqueta!).
- ```pclass``` determina la clase en la que viajaba la persona. Puede ser relevante.
- ```sex``` determina el sexo de la persona. Teniendo en cuenta cuándo naufragó el Titanic, puede ser relevante (de hecho, una primera idea que podemos tener es que se salvaron más mujeres que hombres --esto tendríamos que verlo en un **AED**--).
- ```age``` determina la edad de la persona. Es relevante.
- ```sibsp``` **NO** sabemos a qué se refiere, así que tenemos que esperar a eliminarlo (¡nunca borramos algo que no sabemos qué es!).
- ```parch``` lo mismo que ```sibsp```.
- ```fare``` determina el precio de embarcación, seguramente esté muy relacionado con ```pclass```, pero no deberíamos borrarlo de primeras.
- ```embarked``` determina la inicial de la ciudad de embarque. No debería ser relevante para determinar si la persona murió o no. Como no vamos a predecir datos futuros, vamos a dejar esta variable (podría ser que las personas de una determinada ciudad fueran más resistentes al frío glaciar). Si quisiésemos predecir datos futuros, eliminaríamos esta columna.
- ```class``` parece que contiene la misma información que ```pclass```, la dejamos pendiente de analizar en el **AED**.
- ```who``` parece que contiene la misma información que ```sex```, la dejamos pendiente de analizar en el **AED**.
- ```adult_male``` determina si la persona era hombre adulto. Es recalculada, así que la eliminamos.
- ```deck```. Desconocida.
- ```embark_town``` determina la ciudad de embarque. Repetida (```embarked```). Como las dos son variables de texto, nos quedamos con la más descriptiva. Si una fuese numérica y la otra textual, tendríamos que realizar un **análisis de codificación de variables categóricas** (lo veremos más adelante).
- ```alive``` es la versión booleana de la variable ```survived```. Como ```survived``` es numérica, nos quedamos con ella.
- ```alone``` determina si la persona viajaba sola o no. Nos la quedamos.

Ahora, tenemos que analizar las variables que se han quedado pendientes: ```class``` y ```who``` y eliminar las que sí vamos a borrar: ```adult_male```, ```embarked```, ```alive```

In [3]:
df = df.drop(columns = ["adult_male", "embarked", "alive"])

### Análisis de variables potencialmente eliminables

Hay variables que claramente vamos a borrar, pero hay dos que, en particular, pueden o no contener información relevante: ```class``` y ```who```. Por ello, necesitamos analizarlas un poco más para llegar a una decisión.

**Comenzaremos por ```class```**

Viendo esta variable y comparándola con ```pclass```, parece que ```class``` contiene la información de la clase (primera, segunda y tercera) y ```pclass``` contiene la información de la clase, pero codificada como número.

Veamos si esto es así en realidad. Creamos un subdataset que solo contenga estas dos variables.

In [4]:
_df = df[["pclass", "class"]]
_df.head()

,pclass,class
0,3,Third
1,1,First
2,3,Third
3,1,First
4,3,Third


Ahora vamos a hacer un mapeo de la columna class de forma que se le asigne un valor numérico según la clase (1 para primera, 2 para segunda y 3 para tercera).

In [5]:
_df["class"] = _df["class"].apply(lambda e: 1 if e == "First" else 2 if e == "Second" else 3 if e == "Third" else e) # ¡programación funcional! Uso predicados (funciones) anónimas. Pregúntame si no sabes qué es esto
_df.head()

<ipython-input-5-836aa6e7dd7d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _df["class"] = _df["class"].apply(lambda e: 1 if e == "First" else 2 if e == "Second" else 3 if e == "Third" else e) # ¡programación funcional! Uso predicados (funciones) anónimas. Pregúntame si no sabes qué es esto


,pclass,class
0,3,3
1,1,1
2,3,3
3,1,1
4,3,3


Finalmente, vamos a comprobar si existe una fila para la que ```pclass``` y la nueva ```class``` no coinciden.

In [6]:
_df[_df.pclass != _df["class"]]

,pclass,class


Como no hay ningún resultado, ambas variables describen exactamente el mismo conjunto de datos. Borramos una de ellas (siempre el texto ante el número).

In [7]:
df = df.drop(columns = ["class"])

Ahora vamos a ver las similitudes entre ```sex``` y ```who```.

In [8]:
_df = df[["sex", "who"]]
_df.head()

,sex,who
0,male,man
1,female,woman
2,female,woman
3,female,woman
4,male,man


Como tenemos dos variables de texto diferentes (```male``` vs ```man```, ...), necesitamos convertir esta información en números.

In [9]:
_df["sex"] = _df["sex"].apply(lambda e: 0 if e == "male" else 1 if e == "female" else e)
_df["who"] = _df["who"].apply(lambda e: 0 if e == "man" else 1 if e == "woman" else e)
_df.head()

<ipython-input-9-9911fb290808>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _df["sex"] = _df["sex"].apply(lambda e: 0 if e == "male" else 1 if e == "female" else e)
<ipython-input-9-9911fb290808>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _df["who"] = _df["who"].apply(lambda e: 0 if e == "man" else 1 if e == "woman" else e)


,sex,who
0,0,0
1,1,1
2,1,1
3,1,1
4,0,0


Ahora podríamos comprar línea a línea, como hemos hecho antes (bueno, usando la máscara booleana ```_df.sex``` == ```_df.who```). Sin embargo, el hecho de que la columna ```who``` se haya guardado como un ```float``` cuando nosotros le hemos indicado un ```int``` nos indica que hay algún valor que no se ha modificado (así que ahora hay números y texto).

Lo comprobamos:

In [10]:
_df[_df.sex != _df.who]

,sex,who
7,0,child
9,1,child
10,1,child
14,1,child
16,0,child
...,...,...
831,0,child
850,0,child
852,1,child
869,0,child


Hay 83 filas que no tienen el mismo valor para ```sex``` que para ```who```, que almacenan el valor de si la persona era niño o no.

Por lo tanto, dado que la columna ```who``` mezcla información entre ```sex``` y ```age```, vamos a eliminar esta columna.

Por cierto, ¿a partir de qué edad se considera que una persona deja de ser niño? ¿Cuál es el niño más mayor?

In [11]:
_df = df[["age", "who"]]
_df = _df[_df.who == "child"]
_df.max()

,0
age,15.0
who,child


El niño más mayor tenía 15 años. Reservamos esta información por si nos pudiera ser útil después y borramos la columna ```who```.

In [12]:
df = df.drop(columns = ["who"])
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,deck,embark_town,alone
0,0,3,male,22.0,1,0,7.2500,NaN,Southampton,False
1,1,1,female,38.0,1,0,71.2833,C,Cherbourg,False
2,1,3,female,26.0,0,0,7.9250,NaN,Southampton,True
3,1,1,female,35.0,1,0,53.1000,C,Southampton,False
4,0,3,male,35.0,0,0,8.0500,NaN,Southampton,True


## Valores sin sentido

Habiendo eliminado las columnas inútiles, investigaremos si las restantes tienen valores sin sentido o no.

Para las variables numéricas, tenemos la función ```describe``` que nos dará mucha información.

In [13]:
df.describe()

,survived,pclass,age,sibsp,parch,fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


Como vemos:
- ```survived``` solo tiene dos valores: sí o no. Perfecto.
- ```pclass``` solo tiene tres valores: primera, segunda y tercera clase (vemos que la mayoría de personas viajaban en tercera clase). Perfecto.
- ```age``` la persona más pequeña tenía 0.42 años, la más mayor 80. Puede ser un valor atípico, pero también puede tener sentido. Perfecto.
- Las variables ```sibsp``` y ```parch``` aún no sabemos qué son.
- ```fare``` tiene valores entre 0 (gratis) y 512. Perfecto.

Como no hay ningún valor sin sentido, no tenemos que hacer nada en especial ahora.

Para ver qué hacer con estos valores, imaginemos que no se permitían menores de 18 ni mayores de 65 en el barco.

En este supuesto, sí tenemos información sin sentido (hay menores y jubilados), así que tendríamos que eliminarlos del conjunto de datos.

Para ello, vamos a obtener un subdataset que los elimine con una máscara booleana.

In [14]:
df = df[(df.age >= 18.0) & (df.age <= 65.0)] # recuerda: & es AND
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,deck,embark_town,alone
0,0,3,male,22.0,1,0,7.2500,NaN,Southampton,False
1,1,1,female,38.0,1,0,71.2833,C,Cherbourg,False
2,1,3,female,26.0,0,0,7.9250,NaN,Southampton,True
3,1,1,female,35.0,1,0,53.1000,C,Southampton,False
4,0,3,male,35.0,0,0,8.0500,NaN,Southampton,True


Ahora, todos los valores están ente las edades que hemos indicado:

In [15]:
df[["age"]].describe()

,age
count,593.000000
mean,33.070826
std,11.425805
min,18.000000
25%,24.000000
50%,30.500000
75%,40.000000
max,65.000000


A partir de ahora, podríamos aplicar el resto de técnicas que veremos en los siguientes cuadernillos (las dejamos para otro para no saturar este).

## Valores atípicos (_outliers_)

Los valores atípicos son distintos de los _sin sentido_, dado que este conjunto de valores sí pueden existir, pero pueden ser muy raros y están en la delgada línea entre lo que puede ser real y una mala medida.

Por ello, en general, se suelen tratar siguiendo tres estrategias básicas:
- Estrategia permisiva: se trabaja con los valores atípicos como con el resto.
- Estrategia **NO** permisiva: cualquier valor atípico es eliminado del conjunto de datos.
- Estrategia **semi**permisiva: se extiende el intervalo de aceptación bajo una justificación del analista de datos.

Pero, _¿qué es exáctamente un valor atípico?_

Existen varias formas de definir los datos atípicos. Sin embargo, todas ellas utilizan los valores de los cuartiles (Q)
y del intervalo intercuartílico (IQR), que es la distancia entre el primer y el tercer cuartil (Q3 - Q1).

La primera forma, más restrictiva, establece el intervalo de aceptación desde la media. Siendo $\bar{x}$ la media e IQR la diferencia entre el valor de Q3 y Q1:
$$
[\bar{x} - 1.5 \cdot IQR, \bar{x} + 1.5 \cdot IQR]
$$.

La segunda forma, más laxa, usa los propios cuartiles y los extiende:
$$
[Q1 - 1.5 \cdot IQR, Q3 + 1.5 \cdot IQR]
$$.

Podemos acceder a la información de los cuartiles utilizando la función ```describe```.

In [16]:
df.describe()

,survived,pclass,age,sibsp,parch,fare
count,593.000000,593.000000,593.000000,593.000000,593.000000,593.000000
mean,0.384486,2.177066,33.070826,0.337268,0.317032,35.450674
std,0.486884,0.848695,11.425805,0.579122,0.816219,56.236614
min,0.000000,1.000000,18.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,24.000000,0.000000,0.000000,8.050000
50%,0.000000,2.000000,30.500000,0.000000,0.000000,13.858300
75%,1.000000,3.000000,40.000000,1.000000,0.000000,32.500000
max,1.000000,3.000000,65.000000,3.000000,6.000000,512.329200


El intervalo intercuartícilo se calcula como la diferencia entre el percentil 75 (también llamado Q3) y el percentil 25 (también llamado Q1).

Por ejemplo, para la variable ```age```, tendríamos ```IQR = 40 - 24 = 16```. Posteriormente, esta diferencia se multiplica por 1.5 (ese _1.5 veces_), así que obtendríamos una dilatación total de ```16 * 1.5 = 24```.

Por ello, siguiendo el ejemplo laxo, se considerará que un valor de ```age``` será atípico si está fuera del rango:
$$
[0, 64]
$$

Si considerásemos la versión restrictiva se considerarían valores de ```age``` atípicos aquellos que estuviesen fuera del siguiente invervalo (recuerda, centrado en la media):
$$
[9, 57]
$$

Lo primero que tenemos que hacer antes de decidir qué hacer es conocer cuántos valores atípicos hay.


In [17]:
len(df[df.age > 57]) # longitud del subdataset con la restricción de edad > 57, la más restrictiva

25

Al ser solo 25 valores, podemos eliminarlos sin muchos problemas.

Si hubiesen sido más, podríamos haber aplicado las dos estrategias restantes: trabajar con ellos como con los demás o ser más permisivos en el rango. Esta decisión depende arbitrariamente del analista (y de las pruebas que éste pueda hacer), dado que ya no hay justificación matemática para ninguna de las dos decisiones.

Si quisiésemos borrarlos (o ser más permisivos, la instrucción es la misma), podríamos generar un sub```dataframe``` con los datos en ```age``` **NO** atípicos (usar una **U** otra, no las dos):

In [18]:
len(df[df.age > 64]) # longitud del subdataset con la restricción de edad > 64, la más laxa

3

In [19]:
df = df[df.age <= 64] # estricto
df

,survived,pclass,sex,age,sibsp,parch,fare,deck,embark_town,alone
0,0,3,male,22.0,1,0,7.2500,NaN,Southampton,False
1,1,1,female,38.0,1,0,71.2833,C,Cherbourg,False
2,1,3,female,26.0,0,0,7.9250,NaN,Southampton,True
3,1,1,female,35.0,1,0,53.1000,C,Southampton,False
4,0,3,male,35.0,0,0,8.0500,NaN,Southampton,True
...,...,...,...,...,...,...,...,...,...,...
885,0,3,female,39.0,0,5,29.1250,NaN,Queenstown,False
886,0,2,male,27.0,0,0,13.0000,NaN,Southampton,True
887,1,1,female,19.0,0,0,30.0000,B,Southampton,True
889,1,1,male,26.0,0,0,30.0000,C,Cherbourg,True
